# Clean-up your SageMaker environment

❗ This notebook removes all resources created by all notebooks you run in your environment. The following code cells will:
- permanently delete project or projects you provisioned in your Studio environment
- permanently delete feature group or groups
- permanently delete project-provisioned S3 buckets
- permanently delete objects in S3 buckets under project-related prefixes
- permanently delete inference endpoints

<div class="alert alert-info"> ❗ Code sections with listing of projects, feature groups, and endpoints show resources created outside this workshop. Double check the names of the resources to be deleted so you don't delete your own resources.
</div>

<div class="alert alert-info"> 💡 <strong> This notebook will permanently delete resources in your AWS account. Please double check the names of resources to be deleted! </strong>
</div>

In [ ]:
import sagemaker
import boto3
import time
import json
import os

In [ ]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-start-here notebook   ")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
sm = boto3.client("sagemaker")
s3 = boto3.resource('s3')

## Delete projects

In [ ]:
# Get all projects created by the current domain
projects = [
    {"ProjectName":p["ProjectName"], "ProjectId":p["ProjectId"]} for p in sm.list_projects(MaxResults=100, SortBy="CreationTime")["ProjectSummaryList"] 
        if sm.describe_project(ProjectName=p["ProjectName"])["CreatedBy"]["DomainId"] == domain_id and p["ProjectStatus"] == "CreateCompleted"
]

print(f"These projects have been created by domain {domain_id}: {json.dumps(projects, indent=2)}")

In [ ]:
# Select projects to be deleted
projects_to_delete = []

for p in projects:
    print(f"Are you sure you want to delete this project: {p['ProjectName']}? (y/n)")
    choice = input()
    if choice == 'y':
        projects_to_delete.append(p)
        
print(f"***************************************")
print(f"The following projects will be deleted:\n{json.dumps(projects_to_delete, indent=2)}")
print(f"***************************************")

In [ ]:
for p in projects_to_delete:
    try:
        print(f"Deleting project {p['ProjectName']}:{sm.delete_project(ProjectName=p['ProjectName'])}")
    except Exception:
        pass

## Delete monitoring schedules
You must delete monitoring schedules to be able to successfully remove endpoints deployed by the MLOps projects.

In [ ]:
def delete_mon_schedules(endpoint_name, sm_client):
    print(f"Delete monitoring schedules for the endpoint: {endpoint_name}")
    for s in sm_client.list_monitoring_schedules(EndpointName=endpoint_name)["MonitoringScheduleSummaries"]:
        print(f"Delete monitoring schedule: {s['MonitoringScheduleName']}")
        r = sm_client.delete_monitoring_schedule(MonitoringScheduleName=s['MonitoringScheduleName'])
        print(r)

In [ ]:
for p in projects_to_delete:
    delete_mon_schedules(f"{p['ProjectName']}-staging", sm)
    delete_mon_schedules(f"{p['ProjectName']}-prod", sm)

## Delete CloudFormation stacks
This section deletes the AWS CloudFormation stacks that projects created

In [ ]:
cfn = boto3.client("cloudformation")

for p in projects_to_delete:
    for s in [
            f"sagemaker-{p['ProjectName']}-{p['ProjectId']}-deploy-staging",
            f"sagemaker-{p['ProjectName']}-{p['ProjectId']}-deploy-prod"
            ]:
        try:
            print(f"Delete CloudFormation stack: {s}")
            r = cfn.delete_stack(StackName=s)
            print(r)
            time.sleep(180)
        except Exception as e:
            print(f"Exception in deleting {s}:{e}")
            pass

## Delete feature groups

In [ ]:
feature_groups = sm.list_feature_groups(
    FeatureGroupStatusEquals="Created", 
    SortOrder="Descending", 
    SortBy="CreationTime"
)["FeatureGroupSummaries"]

In [ ]:
feature_groups

In [ ]:
# Select feature groups to be deleted
feature_groups_to_delete = []

for fg in feature_groups:
    print(f"Are you sure you want to delete this feature group: {fg['FeatureGroupName']}? (y/n)")
    choice = input()
    if choice == 'y':
        feature_groups_to_delete.append(fg["FeatureGroupName"])
        
print(f"********************************************")
print(f"The following feature groups will be deleted:\n{json.dumps(feature_groups_to_delete, indent=2)}")
print(f"********************************************")

In [ ]:
def delete_offline_store(feature_group_name: str):
    try:
        offline_store_config = sm.describe_feature_group(FeatureGroupName=feature_group_name)['OfflineStoreConfig']

    except Exception:
        print(f'Feature group: {feature_group_name} does NOT have an offline store!')
        return
    
    offline_store_s3_uri = offline_store_config['S3StorageConfig']['ResolvedOutputS3Uri']
    print(f"all feature store objects under {offline_store_s3_uri} will be deleted!")
    print("Are you sure you want to these objects ? (y/n)")
    
    choice = input()
    if choice == 'y':
        !aws s3 rm {offline_store_s3_uri} --recursive

<div class="alert alert-info"> 💡 <strong> The following code cell will delete the selected feature groups!</strong>
</div>

In [ ]:
for fg in feature_groups_to_delete:
    print(f"Deleting the feature group: {fg}")
    delete_offline_store(fg)
    sm.delete_feature_group(FeatureGroupName=fg)

## Delete project-provisioned S3 buckets

<div class="alert alert-info"> 💡 <strong> The following code cell will delete all S3 buckets created by a project!</strong> If you are in an instructor-led workshop and using a provisioned AWS account, you might be not able to delete any S3 buckets. You can stop here.
</div>

In [ ]:
print(f"*****************************************************")
print(f"The following S3 buckets will be removed permanently!")
print(f"*****************************************************")
for p in projects_to_delete:
    print(f"sagemaker-project-{p['ProjectId']}")

In [ ]:
for p in projects_to_delete:
    !aws s3 rb s3://sagemaker-project-{p['ProjectId']} --force 

## Remove inference endpoints

In [ ]:
endpoints = sm.list_endpoints()["Endpoints"]

In [ ]:
endpoints_to_delete = []

for ep in endpoints:
    print(f"Are you sure you want to delete this endpoint: {ep['EndpointName']}? (y/n)")
    choice = input()
    if choice == 'y':
        endpoints_to_delete.append(ep['EndpointName'])
        
print(f"*********** THESE ENDPOINTS WILL BE DELETED ***********")
print('\n'.join(endpoints_to_delete))
print(f"*******************************************************")

In [ ]:
for ep in endpoints_to_delete:
    try:
        delete_mon_schedules(ep, sm)
        time.sleep(10)
        print(f"Deleting the endpoint: {ep}:{sm.delete_endpoint(EndpointName=ep)}")
    except Exception as e:
        print(f"Exception in deleting {ep}:{e}")
        pass

## Remove project-related objects from the SageMaker S3 data bucket

<div class="alert alert-info"> 💡 <strong> The following code cells will delete all objects under specified S3 prefixes!</strong>
</div>

In [ ]:
prefixes_to_delete = [
    bucket_prefix,
    #s3_fs_query_output_prefix
]

In [ ]:
print(f"************************************************************************")
print(f"All objects under the following S3 prefixes will be removed permanently!")
print(f"************************************************************************")
for p in prefixes_to_delete:
    print(f"{bucket_name}/{p}")

<div class="alert alert-info"> ❗ <strong> Double check the S3 prefix! All data under this prefix will be permanently deleted!</strong>
</div>

In [ ]:
# Un-comment the rm command
for p in prefixes_to_delete:
    # !aws s3 rm s3://{bucket_name}/{p} --recursive
    pass

# Shutdown kernel

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>